In [234]:
import pandas as pd
import numpy as np
from py2neo import Graph
from py2neo import Node, Relationship

In [235]:
data = {'Edge_Primary_Key' : [1, 2, 3, 4, 5, 6, 7, 8], 
        'Edge_Type' : ['Employee','Competitor','Acquired','Has_Financials','NAICS_Code_Subset_of',
                       'Company_NAICS_code','Social_Media_Presence', 'Provides']}
edge_type = pd.DataFrame(data = data)
edge_type

,Edge_Primary_Key,Edge_Type
0,1,Employee
1,2,Competitor
2,3,Acquired
3,4,Has_Financials
4,5,NAICS_Code_Subset_of
5,6,Company_NAICS_code
6,7,Social_Media_Presence
7,8,Provides


In [236]:
data = {'Primary_Key' : [1, 2, 3, 4, 5, 6, 7, 8], 
        'Node_Type' : ['Person','Competing_Company','Acquired_Company','Financials',
                       'Social_Media','NAICS_Code','Products','Services']}
node_type = pd.DataFrame(data = data)
node_type

,Primary_Key,Node_Type
0,1,Person
1,2,Competing_Company
2,3,Acquired_Company
3,4,Financials
4,5,Social_Media
5,6,NAICS_Code
6,7,Products
7,8,Services


In [237]:
node_properties = pd.read_pickle("competitor_node_properties.pkl")
edge_properties = pd.read_pickle("competitor_edge_properties.pkl")

In [238]:
# creating connections table for Neo4j insertion
connections= pd.DataFrame(columns=['Node_Property_Key','Edge_Property_Key','Node_Property_Key_2'])

In [239]:
conn = list(set(edge_properties['Edge_Primary_Key']))
for i in range(len(conn)):
    t = {'Node_Property_Key': [0], 'Edge_Property_Key': [i], 
         'Node_Property_Key_2' : [i+1]}
    temp_conn = pd.DataFrame(data = t)
    connections = connections.append(temp_conn, ignore_index = True)

# Append acquired_date

In [240]:
node_properties_temp = pd.read_pickle("acquisition_node_properties.pkl")
edge_properties_temp = pd.read_pickle("acquisition_edge_properties.pkl")
node_list = list()
primary_comp = node_properties.loc[node_properties['Property'] == 'Cisco Systems', 'Node_Primary_Key'].iloc[0]

count = len(node_properties['Node_Primary_Key'])
for i in range(len(node_properties_temp)):
    node_key = node_properties['Node_Primary_Key'][count-8] + node_properties_temp['Node_Primary_Key'][i] + 1
    t = {'Node_Primary_Key': [node_key], 'Node_Type_Key': [node_properties_temp['Node_Type_Key'][i]], 
         'Property_Name' : [node_properties_temp['Property_Name'][i]], 
         'Property' : [node_properties_temp['Property'][i]]}
    node_list.append(node_key)
    temp_node = pd.DataFrame(data = t)
    node_properties = node_properties.append(temp_node, ignore_index = True)
node_list = list(set(node_list))

count = len(edge_properties['Edge_Primary_Key'])
for i in range(len(edge_properties_temp)):
    key = edge_properties['Edge_Primary_Key'][count-1] + edge_properties_temp['Edge_Primary_Key'][i] + 1
    t = {'Edge_Primary_Key': [key], 
         'Edge_Type_Key': edge_properties_temp['Edge_Type_Key'][i], 
         'Property_Name': edge_properties_temp['Property_Name'][i], 
         'Property': [edge_properties_temp['Property'][i]]}
    c = {'Node_Property_Key': [primary_comp], 'Edge_Property_Key': [key], 
         'Node_Property_Key_2': [node_list[i]]}
    temp_edge = pd.DataFrame(data = t)
    temp_conn = pd.DataFrame(data = c)
    edge_properties = edge_properties.append(temp_edge, ignore_index = True)
    connections = connections.append(temp_conn, ignore_index = True)

# Creating Financial Nodes

In [241]:
node_properties_temp = pd.read_pickle('financials_node_properties_2.pkl')
edge_properties_temp = pd.read_pickle('financials_edge_properties.pkl')
conn = node_properties['Property'][0:17].tolist()
node_list = list()

primary_comp = node_properties.loc[node_properties['Property'] == 'Cisco Systems', 'Node_Primary_Key'].iloc[0]

count = len(node_properties['Node_Primary_Key'])
current_max_nodes = max(node_properties['Node_Primary_Key'])
add_temp = 0
for i in range(len(node_properties_temp)):
    for j in range(len(conn)):
        if conn[j] == node_properties_temp['Node_Primary_Key'][i]:
            add_temp = 1 + j
    node_key = current_max_nodes + add_temp
    node_properties.index[node_properties['Property'] == node_properties_temp['Node_Primary_Key'][i]][0]
    t = {'Node_Primary_Key': [node_key], 'Node_Type_Key': [node_properties_temp['Node_Type_Key'][i]], 
         'Property_Name' : [node_properties_temp['Property_Name'][i]], 
         'Property' : [node_properties_temp['Property'][i]]}
    node_list.append(node_key)
    temp_node = pd.DataFrame(data = t)
    node_properties = node_properties.append(temp_node, ignore_index = True)
node_list = list(set(node_list))

current_max_edges = max(edge_properties['Edge_Primary_Key'])
add_temp = 0
for i in range(len(edge_properties_temp)):
    for j in range(len(conn)):
        if conn[j] == edge_properties_temp['Edge_Primary_Key'][i]:
            prime = j
            add_temp = 1 + j
    key = current_max_edges + add_temp
    t = {'Edge_Primary_Key': [key], 
         'Edge_Type_Key': edge_properties_temp['Edge_Type_Key'][i], 
         'Property_Name': edge_properties_temp['Property_Name'][i], 
         'Property': [edge_properties_temp['Property'][i]]}
    
    c = {'Node_Property_Key': [prime], 'Edge_Property_Key': [key], 
         'Node_Property_Key_2': [node_list[i]]}
    temp_edge = pd.DataFrame(data = t)
    temp_conn = pd.DataFrame(data = c)
    edge_properties = edge_properties.append(temp_edge, ignore_index = True)
    connections = connections.append(temp_conn, ignore_index = True)

# Creating NAICS Nodes and Edges

In [242]:
node_properties_temp = pd.read_pickle("NAICS_node_properties.pkl")
edge_properties_temp = pd.read_pickle('NAICS_edge_properties.pkl')
connections_temp = pd.read_pickle("NAICS_conn_properties.pkl")

In [243]:
#Shifting Nodes
last = len(node_properties['Node_Primary_Key'])
for i in range(len(node_properties_temp)):
    node_properties_temp['Node_Primary_Key'][i] = 1 + node_properties_temp['Node_Primary_Key'][i] + node_properties['Node_Primary_Key'][last-1]
    
for i in range(len(connections_temp)):   
    connections_temp['Node_Property_Key'][i] = 1 + connections_temp['Node_Property_Key'][i] + node_properties['Node_Primary_Key'][last-1]
    connections_temp['Node_Property_Key_2'][i] = 1 + connections_temp['Node_Property_Key_2'][i] + node_properties['Node_Primary_Key'][last-1]

last = len(edge_properties['Edge_Primary_Key'])
for i in range(len(connections_temp)):
    connections_temp['Edge_Property_Key'][i] = 1 + connections_temp['Edge_Property_Key'][i] + edge_properties['Edge_Primary_Key'][last-1]
    edge_properties_temp['Edge_Primary_Key'][i] = 1 + edge_properties_temp['Edge_Primary_Key'][i] + edge_properties['Edge_Primary_Key'][last-1]

last = len(edge_properties['Edge_Primary_Key'])
edge_conn_list = list()
for i in range(len(connections_temp),len(edge_properties_temp)):
    edge_properties_temp['Edge_Primary_Key'][i] = 1 + edge_properties_temp['Edge_Primary_Key'][i] + edge_properties['Edge_Primary_Key'][last-1]
    edge_conn_list.append(edge_properties_temp['Edge_Primary_Key'][i])
    
sub_count = 0
for i in range(262,279):
    node_b = i
    company = node_properties_temp[node_properties_temp['Node_Primary_Key'] == i].reset_index()['Property'][0][:-6]
    node_a = node_properties[node_properties['Property'] == company].reset_index()['Node_Primary_Key'][0]
    
    c = {'Node_Property_Key':[node_a], 'Edge_Property_Key': [edge_conn_list[sub_count]], 'Node_Property_Key_2': [node_b]}
    temp_conn = pd.DataFrame(data = c)
    connections_temp = connections_temp.append(temp_conn, ignore_index = True)
    sub_count+=1

In [244]:
connections = connections.append(connections_temp, ignore_index = True)
node_properties = node_properties.append(node_properties_temp, ignore_index = True)
edge_properties = edge_properties.append(edge_properties_temp, ignore_index = True)

# Competitor CEO

In [245]:
node_properties_temp = pd.read_pickle("people_node_properties.pkl")
edge_properties_temp = pd.read_pickle('people_edge_properties.pkl')
# creating connections table for Neo4j insertion
connections_temp = pd.DataFrame(columns=['Node_Property_Key','Edge_Property_Key','Node_Property_Key_2'])

In [246]:
last = len(node_properties['Node_Primary_Key'])
for i in range(len(node_properties_temp)):
    node_properties_temp['Node_Primary_Key'][i] = 1 + node_properties_temp['Node_Primary_Key'][i] + node_properties['Node_Primary_Key'][last-1]
    
last = len(edge_properties['Edge_Primary_Key'])
for i in range(len(edge_properties_temp)):
    edge_properties_temp['Edge_Primary_Key'][i] = 1 + edge_properties_temp['Edge_Primary_Key'][i] + edge_properties['Edge_Primary_Key'][last-1]

nc = 0 
for i in range(len(edge_properties_temp)):
    c = {'Node_Property_Key':[i], 'Edge_Property_Key': [edge_properties_temp['Edge_Primary_Key'][i]],'Node_Property_Key_2': [node_properties_temp['Node_Primary_Key'][nc]]}
    temp_conn = pd.DataFrame(data = c)
    connections_temp = connections_temp.append(temp_conn, ignore_index = True)
    nc += 3

In [247]:
connections = connections.append(connections_temp, ignore_index = True)
node_properties = node_properties.append(node_properties_temp, ignore_index = True)
edge_properties = edge_properties.append(edge_properties_temp, ignore_index = True)

# Social Media Data

In [248]:
node_properties_temp = pd.read_pickle("node_properties_social_media.pkl")
connections_temp = pd.DataFrame(columns=['Node_Property_Key','Edge_Property_Key','Node_Property_Key_2'])

In [249]:
# creating edge properties table for Neo4j insertion
edge_properties_p = pd.DataFrame(columns = ['Edge_Primary_Key','Edge_Type_Key','Property_Name','Property'])
edge_properties_p

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property


In [250]:
#Append nodes
last_edge = len(edge_properties['Edge_Primary_Key'])
count_conn_edge = 0
edge_list = list()
for i in range(17):
    temp_id = 1 + i + edge_properties['Edge_Primary_Key'][last_edge-1]
    
    e = {'Edge_Primary_Key': [temp_id, temp_id], 'Edge_Type_Key': [7, 7], 'Property_Name': ['Last_queried', 'Query_range(days)'], 
         'Property': ['12/2/2019', '10']}
    temp_edge = pd.DataFrame(data = e)
    edge_properties_p = edge_properties_p.append(temp_edge, ignore_index = True)
    edge_list.append(temp_id)

last = len(node_properties['Node_Primary_Key'])
check = 0
count = 0
node_list = list()
for i in range(len(node_properties_temp)):
    if check < 1:
        temp = 1 + count + node_properties['Node_Primary_Key'][last-1]
        node_list.append(temp)
        check += 1
        count += 1
    else:
        check -= 1
    node_properties_temp['Node_Primary_Key'][i] = temp

for i in range(17):
    c = {'Node_Property_Key':[i], 'Edge_Property_Key': [edge_list[i]],'Node_Property_Key_2': [node_list[i]]}
    temp_conn = pd.DataFrame(data = c)
    connections_temp = connections_temp.append(temp_conn, ignore_index = True)        

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [251]:
extra = pd.read_excel('Social_Media_Followers.xlsx')
e_count = 0
e2_dount = 0
for i in range(len(extra)):
    if (not i % 2) and (i != 0):
        e2_dount += 1
    extra['Node_Primary_Key'][i] = node_list[e2_dount]
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [252]:
node_properties_temp = node_properties_temp.append(extra, ignore_index = True)
node_properties = node_properties.append(node_properties_temp, ignore_index = True)
connections = connections.append(connections_temp, ignore_index = True)
edge_properties = edge_properties.append(edge_properties_p, ignore_index = True)

# Services Nodes

In [253]:
node_properties_temp = pd.read_pickle("services_node_properties.pkl")
edge_properties_temp = pd.read_pickle('services_edge_properties.pkl')
connections_temp = pd.read_pickle("services_conn_properties.pkl")

In [254]:
last = len(node_properties['Node_Primary_Key'])

for i in range(len(node_properties_temp)):
    temp = 1 + node_properties_temp['Node_Primary_Key'][i] + node_properties['Node_Primary_Key'][last-1]
    node_properties_temp['Node_Primary_Key'][i] = temp
    
for i in range(len(connections_temp)):
    temp = 1 + connections_temp['Node_Property_Key_2'][i] + node_properties['Node_Primary_Key'][last-1]
    connections_temp['Node_Property_Key_2'][i] = temp

last = len(edge_properties['Edge_Primary_Key'])
for i in range(len(edge_properties_temp)):
    temp =  1 + edge_properties_temp['Edge_Primary_Key'][i] + edge_properties['Edge_Primary_Key'][last-1]
    edge_properties_temp['Edge_Primary_Key'][i] = temp
    connections_temp['Edge_Property_Key'][i] = temp

In [255]:
node_properties = node_properties.append(node_properties_temp, ignore_index = True)
connections = connections.append(connections_temp, ignore_index = True)
edge_properties = edge_properties.append(edge_properties_temp, ignore_index = True)

# Save Data for Insertion

In [256]:
edge_properties.to_pickle("All_edge_properties.pkl")
node_properties.to_pickle("All_node_properties.pkl")
connections.to_pickle("All_connections.pkl")
node_type.to_pickle("All_node_types.pkl")
edge_type.to_pickle("All_edge_types.pkl")